In [ ]:
import pickle
import json

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
shot_data = []
seasons = list(range(2018,2021))
for year in seasons:
    with open('tables_{}.pkl'.format(year),'rb') as fp:
        tables= pickle.load(fp)
    with open('names_{}.pkl'.format(year),'rb') as fp:
        names= pickle.load(fp)
    stables = []
    for team in tables:
        shoot= tables[team]['shooting']
        shoot_s = shoot[shoot.MP > 850].iloc[:,7:12]
        stables.append(shoot_s)
    shot_dist = pd.concat(stables)
    shot_dist['season'] = year
    shot_data.append(shot_dist)
shot_data = pd.concat(shot_data)

In [ ]:
data = json.load(open('real-player.json','rb'))
df = pd.DataFrame(data['ratings'])
df = df.drop(['fuzz','abbrev_if_new_row'],1)#.set_index(['slug','season'])
df = df.set_index(['slug','season']).reset_index()
df

In [ ]:
const = {
  "shootingAtRim":{"ratings":["hgt","stre","dnk","oiq"],"weights":[2,0.3,0.3,0.2]},
  "shootingLowPost":{ 
    "ratings":["hgt","stre","spd","ins","oiq"],
    "weights":[1,0.6,0.2,1,0.4],
    "skill":{"label":"Po","cutoff":0.61}
  },
  "shootingMidRange":{"ratings":["oiq","fg","stre"],"weights":[-0.5,1,0.2]},
  "shootingThreePointer":{
    "ratings":["oiq","tp"],
    "weights":[0.1,1],
    "skill":{"label":"3","cutoff":0.59}
  }
}

In [ ]:
df = df[df.season.isin(seasons)].copy()

In [ ]:
shots = {k:[] for k in const.keys()}

In [ ]:
dfc = list(df.columns)
for row in df.itertuples():
    for k,v in const.items():
        value = 0
        weights = 0
        for r,w in zip(v['ratings'],v['weights']):
            value += row[dfc.index(r)+1]*w
            weights +=w
        shots[k].append(np.clip((value/weights)/100,0,1))

In [ ]:
for k,v in shots.items():
    df[k] = v

In [ ]:
df

In [ ]:
dataset = df.copy()
dcol = list(dataset.columns)
dcol[0] = 'Name'
dataset.columns = dcol
dataset = dataset.set_index(['Name','season'])
shot_datas = shot_data.reset_index().set_index(['Name','season'])
dataset = dataset.join(shot_datas,how='outer').dropna().reset_index()

In [ ]:
X = []
Y = []
groups = [(['shootingAtRim'],['0-3']),
 (['shootingLowPost'],['3-10']),
 (['shootingMidRange'],['10-16','16-3P']),
 (['shootingThreePointer'],['3P'])]

rate = list(df.columns[2:17])

groups2= [(rate,['0-3']),
 (rate,['3-10']),
 (rate,['10-16','16-3P']),
 (rate,['3P'])]


for g in groups:
    X.append(np.array(dataset[g[0]]))
    Y.append(np.array(dataset[g[1]].sum(1)))
X = np.hstack(X).astype(np.float32)
Y = np.array(Y).astype(np.float32).T

# remove the dups
if X.shape[1] > 20:
    X = X[:,:15]/100

In [ ]:
X.shape,Y.shape

In [ ]:
import torch


In [ ]:
def softXEnt (input, target):
    logprobs = torch.nn.functional.log_softmax (input, dim = 1)
    return  -(target * logprobs).sum() / input.shape[0]

def LogLoss(input,target):
    return -(target*torch.log(input)).sum()

D_in,D_out = X.shape[1],Y.shape[1]

Xt = torch.tensor(X)
Yt = torch.tensor(Y)

w1 = torch.randn(D_in, D_out,requires_grad=True)

b0 = torch.ones((D_in),requires_grad=True)

b1 = torch.randn((D_out),requires_grad=True)
b2 = torch.randn((D_in),requires_grad=True)
b3 = torch.randn((D_in),requires_grad=True)
b4 = torch.zeros((D_out),requires_grad=True)

m = torch.nn.Softmax(1)
m3 = torch.nn.Tanh()
m4 = torch.nn.ReLU()

H = 4
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.Tanh(),
    torch.nn.Linear(H, D_out),
    #torch.nn.Tanh(),
    #torch.nn.Linear(H, D_out),
)

criterion = LogLoss# torch.nn.MSELoss() # LogLoss#
optimizer = torch.optim.Adam([w1,b1,b0,b2,b3,b4]+list(model.parameters()), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.1, patience=200, threshold_mode='rel')
#Xt[:,2]  = 0 #mid-ranger missing

for t in range(20000):
    vec  =model(Xt)
    #vec = (b0*(Xt))+b1 + m3(b2*Xt+b3)#  torch.matmul(Xt+b1,w1)
    #vec =m4(torch.matmul(Xt,w1)+b4) #  m4(b0*Xt+b1) + m3(b2*Xt+b3) + 
    #vec[:,2] = 0 # mid-range is leftovers
    y_pred = m(vec)

    # Compute and print loss.
    loss = criterion(y_pred, Yt) 
    p1 = torch.cat([x.view(-1) for x in model.parameters()])

    if (t % 1000) == 0:
        print(t, loss.item(),1e-3*torch.norm(p1, 1).item())
    loss += 1e-3*torch.norm(p1, 1)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step(loss)

In [ ]:
m2 = torch.nn.MSELoss()
m2(y_pred,Yt)

In [ ]:
vec

In [ ]:
y_pred

In [ ]:
Yt

In [ ]:
torch.set_printoptions(precision=3,sci_mode=False)

In [ ]:
print(b1)

In [ ]:
print(w1)

In [ ]:
y_pred

In [ ]:
w1[:,3]

In [ ]:
for i,row in enumerate(dataset.itertuples()):
    if row[1] in ['curryst01','hardeja01']: 
        print(i,Y[i],y_pred[i])

In [ ]:
for i,row in enumerate(dataset.itertuples()):
    if row[1] in ['goberru01','jordade01']:
        print(i,Y[i],y_pred[i])

In [ ]:
for i,row in enumerate(dataset.itertuples()):
    if row[1] in ['derozde01','bookede01','paulch01']:
        print(i,Y[i],y_pred[i])

In [ ]:
plt.hist(dataset.shootingThreePointer)

In [ ]:
list(model.parameters())

In [ ]:
for m in model.parameters():
    #print(m.shape)
    print(list(m.detach().numpy().T.ravel()),';')

In [ ]:
m.detach().numpy()

In [ ]:
Xt[0]

In [ ]:
vec[0]

In [ ]:
y_pred[0]

In [ ]:
a = [0]
a = np.array(a)
np.array(a).mean(),np.array(a).std()

In [ ]:
plt.hist(a)

In [ ]:
a2 = 17.6*0.25*(a-0.87)
plt.hist(a2)
plt.title(a2.mean())
a2.std()

In [ ]:
a3 = 0.5+1/(1+np.exp(-a2))
plt.hist(a3)
a3.mean(),a3.std()